## Import some packages

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from scipy import integrate
import sys
import os
sys.path.append("../../../../")
from ADFWI.propagator  import *
from ADFWI.model       import *
from ADFWI.view        import *
from ADFWI.utils       import *
from ADFWI.survey      import *

project_path = "./examples/pip_usage/data/elastic"
if not os.path.exists(os.path.join(project_path,"model")):
    os.makedirs(os.path.join(project_path,"model"))
if not os.path.exists(os.path.join(project_path,"waveform")):
    os.makedirs(os.path.join(project_path,"waveform"))
if not os.path.exists(os.path.join(project_path,"survey")):
    os.makedirs(os.path.join(project_path,"survey"))

## Basic Parameter

In [2]:
device = "cuda:1"         # Specify the GPU device
dtype = torch.float32     # Set data type to 32-bit floating point
ox, oz = 0, 0             # Origin coordinates for x and z directions
nz, nx = 78, 180          # Grid dimensions in z and x directions
dx, dz = 45, 45           # Grid spacing in x and z directions
nt, dt = 1600, 0.003      # Time steps and time interval
nabc = 50                 # Thickness of the absorbing boundary layer
f0 = 3                    # Initial frequency in Hz
free_surface = True       # Enable free surface boundary condition


## Define the True Velocity Model

In [ ]:
# Load the Marmousi model dataset
marmousi_model = load_marmousi_model(in_dir="/ailab/user/liufeng1/project/04_Inversion/ADFWI-github/examples/datasets/marmousi2_source")
x         = np.linspace(5000, 5000+dx*nx, nx)
z         = np.linspace(0, dz*nz, nz)
vel_model = resample_marmousi_model(x, z, marmousi_model)
vp        = vel_model['vp'].T
vs        = vel_model['vs'].T
rho       = np.power(vp, 0.25) * 310

# Initialize the elastic model with parameters and properties
model = IsotropicElasticModel(
                    ox,oz,nx,nz,dx,dz,
                    vp,vs,rho,
                    vp_grad = False,vs_grad = False, 
                    free_surface=free_surface,
                    abc_type="PML",abc_jerjan_alpha=0.007,
                    auto_update_rho=False,
                    auto_update_vp =False,
                    nabc=nabc,
                    device=device,dtype=dtype)

# Save the model to a file
model.save(os.path.join(project_path, "model/true_model.npz"))

# Print the model representation
print(model.__repr__())

In [ ]:
# Plot the primary wave velocity (vp) and density (rho) of the model
model._plot_vp_vs_rho(figsize=(12,5),wspace=0.2,cbar_pad_fraction=0.18,cbar_height=0.04,cmap='coolwarm',save_path=os.path.join(project_path,"model/true_vp_vs_rho.png"))

## Define the observed System： Survey = Source + Receiver

In [5]:
# Define source positions in the model
src_z = np.array([10   for i in range(2,nx-2,5)]) 
src_x = np.array([i    for i in range(2,nx-2,5)])
# Generate wavelet for the source
src_t, src_v = wavelet(nt, dt, f0, amp0=1)  # Create time and wavelet amplitude
src_v = integrate.cumtrapz(src_v, axis=-1, initial=0)  # Integrate wavelet to get velocity

source = Source(nt=nt, dt=dt, f0=f0)  # Initialize source object

# Method 1: Add multiple sources at once (commented out)
# source.add_sources(src_x=src_x, src_z=src_z, src_wavelet=src_v, src_type='mt', src_mt=np.array([[1,0,0],[0,1,0],[0,0,1]]))

# Method 2: Loop through each source position to add them individually
for i in range(len(src_x)):
    source.add_source(src_x=src_x[i], src_z=src_z[i], src_wavelet=src_v, src_type="mt", src_mt=np.array([[1,0,0],[0,1,0],[0,0,1]]))


In [6]:
# Define receiver positions in the model
rcv_z = np.array([10  for i in range(0,nx,1)])
rcv_x = np.array([j   for j in range(0,nx,1)])
receiver = Receiver(nt=nt, dt=dt)  # Initialize receiver object

# Method 1: Add all receivers at once (commented out)
# receiver.add_receivers(rcv_x=rcv_x, rcv_z=rcv_z, rcv_type='pr')

# Method 2: Loop through each receiver position to add them individually
for i in range(len(rcv_x)):
    receiver.add_receiver(rcv_x=rcv_x[i], rcv_z=rcv_z[i], rcv_type="pr")

In [ ]:
# Create a survey object using the defined source and receiver
survey = Survey(source=source, receiver=receiver)

# Print a representation of the survey object to check its configuration
print(survey.__repr__())

In [ ]:
# Plot the wavelet used in the source
source.plot_wavelet(save_path=os.path.join(project_path, "survey/wavelets.png"))

In [ ]:
# Plot the survey configuration over the velocity model
survey.plot(model.vp, cmap='coolwarm', save_path=os.path.join(project_path, "survey/observed_system.png"))

## Define the propagator & Forward Modeling

In [10]:
# Initialize the wave propagator using the specified model and survey configuration
F = ElasticPropagator(model,survey,device=device)

In [ ]:
# Retrieve the damping array from the propagator and plot it to visualize boundary conditions
if model.abc_type == "PML":
    bcx = F.bcx
    bcz = F.bcz
    title_param = {'family':'Times New Roman','weight':'normal','size': 15}
    plot_bcx_bcz(bcx,bcz,dx=dx,dz=dz,wspace=0.25,title_param=title_param,cbar_height=0.04,cbar_pad_fraction=-0.05,save_path=os.path.join(project_path,"model/boundary_condition.png"))
else:
    damp = F.damp
    plot_damp(damp)

In [12]:
# Perform the forward propagation to record waveforms
record_waveform = F.forward(fd_order=4)

# Extract recorded pressure wavefield and particle velocities
rcv_txx = record_waveform["txx"]
rcv_tzz = record_waveform["tzz"]
rcv_txz = record_waveform["txz"]
rcv_vx  = record_waveform["vx"]
rcv_vz  = record_waveform["vz"]

# Extract forward wavefields for analysis
forward_wavefield_txx = record_waveform["forward_wavefield_txx"]
forward_wavefield_tzz = record_waveform["forward_wavefield_tzz"]
forward_wavefield_txz = record_waveform["forward_wavefield_txz"]
forward_wavefield_vx  = record_waveform["forward_wavefield_vx"]
forward_wavefield_vz  = record_waveform["forward_wavefield_vz"]

In [13]:
# Create a SeismicData object to store observed data from the survey
d_obs = SeismicData(survey)

# Record the waveform data into the SeismicData object
d_obs.record_data(record_waveform)

# Save the recorded data to a specified file
d_obs.save(os.path.join(project_path, "waveform/obs_data.npz"))

## Visulization the Synthetic Waveform

In [ ]:
normalize = False  # Set normalization for waveform plotting

# Loop over shots to plot the observed waveforms
for i_shot in range(1):  # Currently set to plot only the first shot
    show = (i_shot == 0)  # Show the plot only for the first shot

    # Plot 2D waveform for the specified shot
    d_obs.plot_waveform2D(
        i_shot=i_shot,
        rcv_type="pressure",
        acoustic_or_elastic="elastic",
        normalize=normalize,
        figsize=(6, 6),
        cmap='coolwarm',
        save_path=os.path.join(project_path, f"waveform/obs_2D_shot_{i_shot}.png"),
        show=show
    )

    # Plot wiggle representation of the waveform for the specified shot
    d_obs.plot_waveform_wiggle(
        i_shot=i_shot,
        rcv_type="pressure",
        acoustic_or_elastic="elastic",
        normalize=normalize,
        save_path=os.path.join(project_path, f"waveform/obs_wiggle_shot_{i_shot}.png"),
        show=show
    )

In [ ]:
# Plot the waveform trace for a specific shot and trace index
d_obs.plot_waveform_trace(
    i_shot=0,  # Index of the shot to plot (first shot)
    i_trace=10,  # Index of the trace to plot (10th trace)
    acoustic_or_elastic='elastic',
    normalize=True  # Normalize the waveform for better visualization
)